In [ ]:
try:
    with open("global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [ ]:
import random
import tensorflow as tf
import numpy as np
import re
import pprint
from src.wikipedia import Wikipedia
#random.seed(12345)

In [ ]:
def distortText(text, swap_type = "symboldelete"):
       
    # Distortion types
    distortions_symbols = ["symbolswap","symboldelete","symbolreplacerandom","symbolreplaceprev"]
    distortions_words = ["wordtrimright", "wordswapprev"]
    distortions_sentences = ["deletespaces"]
    # Symbol operations: max 2 symbols per word, max 2 symbol distortion types per word
    # Word trim: max 1 per word
    # Delete spaces: max all spaces, min 0 spaces
    
    list_words = text.split()
    
    distortion_prob_symbol = 0.5
    for i in range(len(list_words)):
        r = random.random()
        if swap_type == "symbolswap":
            # SYMBOL SWAP
            ## min distorted symbol = 0, max distorted symbol = len-2 because of indexing with 0 and this is the 1st swappable symbol
            distorted_symbol = round((len(list_words[i]) - 2) * random.random())
            s1 = list_words[i][distorted_symbol + 1]
            #print("{}---{}-{}-{}-{}".format(distorted_symbol, list_words[i][0:distorted_symbol - 1], list_words[i][distorted_symbol], s1, list_words[i][distorted_symbol + 1:]))
            #print(list_words[i][0:(distorted_symbol - 1) * (distorted_symbol > 0)])
            list_words[i] = list_words[i][0:distorted_symbol] + s1 + list_words[i][distorted_symbol] + list_words[i][distorted_symbol + 2:]
        elif swap_type == "symboldelete":
            # SYMBOL DELETE
            ## min distorted symbol = 0, max distorted symbol = len-1
            distorted_symbol = round((len(list_words[i]) - 1) * random.random())
            list_words[i] = list_words[i][0:distorted_symbol] + list_words[i][distorted_symbol + 1:]
        elif swap_type == "symbolreplacerandom":
            symbols = "abcdefghijklmnopqrstuvwxyz"
            # SYMBOL REPLACE RANDOM
            ## min distorted symbol = 0, max distorted symbol = len-1
            distorted_symbol = round((len(list_words[i]) - 1) * random.random())
            replace = symbols[round((len(symbols) - 1) * random.random())]
            list_words[i] = list_words[i][0:distorted_symbol] + replace + list_words[i][distorted_symbol + 1:]
        elif swap_type == "symbolreplaceprev":
            # SYMBOL REPLACE PREVIOUS
            ## min distorted symbol = 1, max distorted symbol = len - 1
            distorted_symbol = round(1 + (len(list_words[i]) - 2) * random.random())
            #print("{} - {}".format(distorted_symbol, list_words[i]))
            #print("{}---{}-{}-{}".format(distorted_symbol, list_words[i][0:(distorted_symbol)], list_words[i][distorted_symbol - 1], list_words[i][distorted_symbol + 1 * (len(list_words[i]) > distorted_symbol):distorted_symbol + 1 * (len(list_words[i]) > distorted_symbol)]))
            list_words[i] = list_words[i][0:(distorted_symbol)] + list_words[i][distorted_symbol - 1] + list_words[i][distorted_symbol + 1:]
        
        if swap_type == "wordtrimright":
            # WORD TRIM RIGHT
            ## min symbol = 1, max distorted symbol = len - 1
            trim_from = round(1 + (len(list_words[i]) - 1) * random.random())
            list_words[i] = list_words[i][:trim_from]
    
    if swap_type == "wordswapprev":
        # WORD SWAP
        word_id = round((len(list_words) - 2) * random.random())
        s1 = list_words[word_id]
        list_words[word_id] = list_words[word_id + 1]
        list_words[word_id + 1] = s1
        
    if swap_type == "deletespaces":
        # DELETE WHITESPACES
        ## -2 because there are len - 1 spaces in total
        space_id = round((len(list_words) - 2) * random.random())
        list_words[space_id] = list_words[space_id] + list_words[space_id + 1]
        list_words.pop(space_id + 1)
            
    distortedText = " ".join(list_words)
    return distortedText

In [ ]:
text = "Antons likes pizza"
print(distortText(text=text,swap_type="symbolswap"))

print(distortText(text=text,swap_type="symboldelete"))

print(distortText(text=text,swap_type="symbolreplacerandom"))

print(distortText(text=text,swap_type="symbolreplaceprev"))

print(distortText(text=text,swap_type="wordtrimright"))

print(distortText(text=text,swap_type="wordswapprev"))

print(distortText(text=text,swap_type="deletespaces"))

In [ ]:
wikipedia = Wikipedia(
    language="simple",
    cache_directory_url=False
)

In [ ]:
alphabets = {'a' : 0, 'b': 1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7, 'i':8, 'j':9, 'k':10, 'l':11, 'm':12, 'n':13, 'o':14,
            'p':15, 'q':16, 'r':17, 's':18, 't':19, 'u':20, 'v':21, 'w':22, 'x':23, 'y':24, 'z':25, 
            '0':26, '1':27, '2':28, '3':29, '4':30, '5':31, '6':32, '7':33, '8':34, '9':35, 
            ' ':36, ',':37, '.':38, ':':39, ';':40, '"':41, "'":42, '':43} #43 = unknown symbol

idxs = [alphabets[ch] if ch in alphabets else 43 for ch in 'az 123#']

idxs

In [ ]:
one_hot = tf.one_hot(idxs, depth=len(alphabets), dtype=tf.uint8)

In [ ]:
sess = tf.InteractiveSession()
one_hot.eval()

In [ ]:
# Cleaning up simple wikipedia texts
pattern_ignored_words = re.compile(
    r"""
    (?:(?:thumb|thumbnail|left|right|\d+px|upright(?:=[0-9\.]+)?)\|)+
    |^\s*\|.+$
    |^REDIRECT\b""",
    flags=re.DOTALL | re.UNICODE | re.VERBOSE | re.MULTILINE)
pattern_new_lines = re.compile('[\n\r ]+', re.UNICODE)
texts = [wikipedia.documents[i].text for i in range(len(wikipedia.documents))]
texts = [pattern_ignored_words.sub('', texts[i]) for i in range(len(texts))]
texts = [pattern_new_lines.sub(' ', texts[i]) for i in range(len(texts))]
texts = [texts[i].replace("\\", "") for i in range(len(texts))]

In [ ]:
print(texts[0])

In [ ]:
# Simple wikipedia article texts into single sentences
sentences += [texts[i].split(". ") for i in range(len(texts))]

In [ ]:
print(sentences[10000:10020])